In [ ]:
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd '/content/drive/MyDrive/cs685project'

/content/drive/MyDrive/cs685project


In [ ]:
%ls

sample_data/


In [ ]:
!pip install transformers==4.35.0

In [ ]:
!pip install -q trl

In [ ]:
!pip install -U flash-attn --no-build-isolation

In [ ]:
!pip install peft


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 4.0 MB/s eta 0:00:00


In [ ]:
!pip install huggingface_hub


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) Y
Token is valid (permission: read).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your terminal in case you want to set the 'store' cre

In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer
from peft import LoraConfig, PeftModel, AutoPeftModelForCausalLM
import argparse
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
import time
import torch
import os

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="data/chat_gpt_questions/wikimia.jsonl")

dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'label', 'id', 'question', 'answer', 'prediction'],
        num_rows: 776
    })
})

In [ ]:
dataset = dataset.rename_column("label", "mia_label")
dataset = dataset.rename_column("answer", "label")

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()

In [ ]:
def formatting_prompts_func(examples):
    output_text = []
    for i in range(len(examples["input"])):
        instruction = "Answer the following question"
        input_text = examples["question"][i]
        response = examples["answer"][i]
        text = f'''Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

        ### Instruction:
        {instruction}

        ### Input:
        {input_text}

        ### Response:
        {response}
        '''
        if len(text.split())<150:
            output_text.append(text)
    return output_text

In [ ]:
model_name = "EleutherAI/pythia-1.4b"
lora_r = 8
lora_alpha = 16
lora_dropout = 0.05
t0=time.time()
# Load base model
model = AutoModelForCausalLM.from_pretrained(model_name,torch_dtype=torch.bfloat16, use_flash_attention_2=True,cache_dir="./hf_cache")
model = model.to("cuda")
model.config.use_cache = False
model.config.pretraining_tp = 1
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./hf_cache", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training
print(f"Loading model and tokenizer took {time.time() - t0} seconds")
#define the training arguments
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)
#define the trainer arguments
num_train_epochs = 3
per_device_train_batch_size = 3
gradient_accumulation_steps = 128
optim = "adamw_torch_fused"
lr_scheduler_type = "cosine"
#max_steps = (50000*3)//1024
max_steps = -1
warmup_ratio = 0.03
save_steps = 0
learning_rate = 3e-4
weight_decay = 0.001
logging_steps = 25
fp16 = False
bf16 = False
max_grad_norm = 0.3
group_by_length = True
# Set training parameters
training_arguments = TrainingArguments(
    output_dir="tmp",
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    # report_to="tensorboard"
)
# Maximum sequence length to use
max_seq_length = 128
# Pack multiple short examples in the same input sequence to increase efficiency
packing = False
# response_template_with_context = "### Response:"  # We added context here: "\n". This is enough for this tokenizer
# response_template_ids = tokenizer.encode(response_template_with_context, add_special_tokens=False)[:]  # Now we have it like in the dataset texts: `[2277, 29937, 4007, 22137, 29901]`
# collator = DataCollatorForCompletionOnlyLM(response_template_ids, tokenizer=tokenizer)
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    peft_config=peft_config,
    #formatting_func=formatting_prompts_func,
    dataset_text_field="input",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)
model, dataset, trainer =  accelerator.prepare(model, dataset, trainer)

trainer.train()
trainer.model.save_pretrained('pythia-1.4b')
del model
torch.cuda.empty_cache()
model = AutoPeftModelForCausalLM.from_pretrained("pythia-1.4b", device_map="auto", torch_dtype=torch.bfloat16)
model = model.merge_and_unload()
output_merged_dir = os.path.join("pythia-1.4b", "final_merged_checkpoint")
model.save_pretrained(output_merged_dir, safe_serialization=True)

You are attempting to use Flash Attention 2.0 with a model initialized on CPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
